# Imports

In [1]:
#Creating a clean Slate
from IPython import get_ipython
get_ipython().magic('reset -sf')
#Directing to the correct location and getting the file
import os
os.chdir('C:/Users/ritux/OneDrive - Danmarks Tekniske Universitet/Skrivebord/3 -Computational Tools for Data Science/')
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

C:\Users\ritux\AppData\Local\Temp\ipykernel_13132\2114608133.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


# Support Functions

In [2]:
headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
            'Accept-Language': 'en-US,en;q=0.9',
            'Referer': 'https://www.google.com/'  # Sometimes needed to bypass blocks
        }

## Scraping

In [3]:
def url_scrapping(base_url,limit,headers=headers):

    urls = [] # List of urls

    to_visit = [base_url] # Queue
    while to_visit and len(urls) < limit:
        
        # Getting an URL form the queue
        current_url = to_visit.pop(0)
        # Issuing the request for access to the webpage
        response = requests.get(current_url, headers=headers)
        # Ensuring we are successful in the request
        if response.status_code != 200:
            print(f"Request failed with status code {response.status_code} at url: {current_url}")
        
        # Obtaining the information in the webpage
        Soup = BeautifulSoup(response.text, 'html.parser')
        # Iterating through the linked pages from the page we are visiting
        for a in Soup.find_all('a', href=True): 
            url = urljoin(base_url, a['href'])
            # Ensuring it stays within the chosen page
            if url.startswith(base_url) and url: 
                # Keeping only new urls 
                if url not in urls:
                    urls.append(url)
                    # Adding to queue only unscheduled urls
                    if url not in to_visit: 
                        to_visit.append(url)
    return urls


## Filtering

In [4]:
def filter_urls(urls, limit,condition,headers=headers):
    filtered_urls = []
    for url in urls:
        try:
            # Fetch the page
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"Request failed with status code {response.status_code} at url: {url}")
            
            soup = BeautifulSoup(response.text, 'html.parser')

            # Check if the page contains the ingredient selector
            if soup.select(condition) and url not in filtered_urls:
                filtered_urls.append(url)
                
                # Stop if we've reached the limit
                if len(filtered_urls) >= limit:
                    break
        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")
            continue  # Skip to the next URL in case of an error

    return filtered_urls


In [5]:
# Function to filter URLs
def bcc_filter_urls(urls, limit,headers=headers):
    filtered_urls = []
    for url in urls:
        segments = url.split('/')
        if url.startswith('https://www.bbc.co.uk/') and url.startswith('https://www.bbc.co.uk/food/recipes') and len(segments) in [6,7]:
            try:
                response = requests.get(url,headers=headers)
                if response.status_code != 200:
                    print(f"Request failed with status code {response.status_code} at url: {url}")
            
                soup = BeautifulSoup(response.text, 'html.parser')
                # Check if the page contains the ingredients
                if soup.select_one("div.recipe-ingredients-wrapper ul.recipe-ingredients__list li.recipe-ingredients__list-item a.recipe-ingredients__link")  and url not in filtered_urls:
                    filtered_urls.append(url)
                    if len(filtered_urls) >= limit:
                        break
            except requests.RequestException as e:
                print(f"Error fetching {url}: {e}")
                continue  # Skip to the next URL in case of an error

    return filtered_urls

## Ingredients

In [6]:
# Function to extract ingredients from BBC website
def bbc_get_ingredients(urls):
    ingredients = []
    for url in urls:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.select_one("h1.content-title__text").text.strip()
        recipe_ingredients = []
        # Use soup.select with the CSS selector for ingredients
        for item in soup.select("div.recipe-ingredients-wrapper ul.recipe-ingredients__list li.recipe-ingredients__list-item a.recipe-ingredients__link"):
            name = item.text.strip()
            # Combine to form a full ingredient line
            recipe_ingredients.append({'title': title,'description': name})
        ingredients.append(recipe_ingredients)
    return ingredients

In [7]:
def all_get_ingredients(urls):
    ingredients = []
    for url in urls:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.select_one("title").text.strip()
        recipe_ingredients = []
        # Use soup.select with the CSS selector for ingredients
        for item in soup.select("ul.mm-recipes-structured-ingredients__list > li.mm-recipes-structured-ingredients__list-item"):
            name = item.select_one("span[data-ingredient-name]").get_text(strip=True)
            # Combine to form a full ingredient line
            recipe_ingredients.append({'title': title,'description': name})
        ingredients.append(recipe_ingredients)
    return ingredients

In [8]:
def two_get_ingredients(urls):
    ingredients = []
    for url in urls:
        # Fetch the page content
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.select_one("title").text.strip()
        recipe_ingredients = []
        # Use soup.select with the CSS selector for ingredients
        for item in soup.select(".wprm-recipe-ingredient"):
            # Get name if they exist
            name = item.select_one(".wprm-recipe-ingredient-name")
            if name:# Add to ingredients list
                name = name.get_text(strip=True)
                # Combine to form a full ingredient line
                recipe_ingredients.append({'title': title,'description': name})
                
        ingredients.append(recipe_ingredients)
    return ingredients

# Body

In [9]:
bbc_url = 'https://www.bbc.co.uk/food/recipes'
all_url = "https://www.allrecipes.com"
two_url = 'https://www.twopeasandtheirpod.com/'

bbc_scraped_urls = url_scrapping(bbc_url,1000)
all_scraped_urls = url_scrapping(all_url,1000)
two_scraped_urls = url_scrapping(two_url,1000)


Request failed with status code 403 at url: https://www.twopeasandtheirpod.com/wp-login.php?redirect_to=https%3A%2F%2Fwww.twopeasandtheirpod.com%2Fsaved-recipes%2F


In [10]:
all_con = "li.mm-recipes-structured-ingredients__list-item"
two_con = ".wprm-recipe-ingredient"

bbc_url_filterd = bcc_filter_urls(bbc_scraped_urls, 1000,headers=headers)
all_url_filterd = filter_urls(all_scraped_urls, 1000,all_con,headers=headers)
two_url_filterd = filter_urls(two_scraped_urls, 1000,two_con,headers=headers)


Request failed with status code 404 at url: https://www.allrecipes.com/realms/alrcom/broker/facebook/login?client_id=alrcom&tab_id=swrShh18Yq4&session_code=HWYwSygV2-iLyaAFvoDFm6hNI-_Ohbgz80ojxcQM_QQ
Request failed with status code 404 at url: https://www.allrecipes.com/realms/alrcom/broker/google/login?client_id=alrcom&tab_id=swrShh18Yq4&session_code=HWYwSygV2-iLyaAFvoDFm6hNI-_Ohbgz80ojxcQM_QQ
Request failed with status code 404 at url: https://www.allrecipes.com/realms/alrcom/login-actions/registration?client_id=alrcom&tab_id=swrShh18Yq4
Request failed with status code 404 at url: https://www.allrecipes.com/realms/alrcom/broker/facebook/login?client_id=alrcom&tab_id=qE8HMqXMlvI&session_code=a9e2BsNbGyZyiir1CYlNxRFpNfM8mYg8cB4EvlmtLAc
Request failed with status code 404 at url: https://www.allrecipes.com/realms/alrcom/broker/google/login?client_id=alrcom&tab_id=qE8HMqXMlvI&session_code=a9e2BsNbGyZyiir1CYlNxRFpNfM8mYg8cB4EvlmtLAc
Request failed with status code 404 at url: https://www.

C:\Users\ritux\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Request failed with status code 403 at url: https://www.twopeasandtheirpod.com/wp-login.php/


In [11]:
two_ingredients = two_get_ingredients(two_url_filterd)
all_ingredients = all_get_ingredients(all_url_filterd)
bcc_ingredients = bbc_get_ingredients(bbc_url_filterd)

## Concatenate the three websites

In [12]:
print("For bbc:")
print("Number of URLs found :" , len(bbc_scraped_urls))
print("Number of URLs filtered:" , len(bbc_url_filterd))
print("Number of URLs with recipes:" , len(bcc_ingredients))
n_bcc =len(bcc_ingredients)
print("\n")
print("For all:")
print("Number of URLs found :" , len(all_scraped_urls))
print("Number of URLs filtered:" , len(all_url_filterd))
print("Number of URLs with recipes:" , len(all_ingredients))
n_all =len(all_ingredients)
print("\n")
print("For two:")
print("Number of URLs found :" , len(two_scraped_urls))
print("Number of URLs filtered:" , len(two_url_filterd))
print("Number of URLs with recipes:" , len(two_ingredients))
n_two =len(two_ingredients)



For bbc:
Number of URLs found : 1010
Number of URLs filtered: 552
Number of URLs with recipes: 552


For all:
Number of URLs found : 1311
Number of URLs filtered: 575
Number of URLs with recipes: 575


For two:
Number of URLs found : 1025
Number of URLs filtered: 386
Number of URLs with recipes: 386


In [13]:
count  = 0
all_ingredients_df = pd.DataFrame(columns=['id','source','link', 'title'])

# Extract ingredients
for n in range(n_bcc):
    count +=1
    ingredients_df = pd.DataFrame(bcc_ingredients[n])
    ingredients_df['id'] = count
    ingredients_df['link'] = bbc_url_filterd[n]
    ingredients_df['source'] = 'bbc'
    all_ingredients_df = pd.concat([all_ingredients_df, ingredients_df], ignore_index=True)

for n in range(n_all):
    count +=1
    ingredients_df = pd.DataFrame(all_ingredients[n])
    ingredients_df['id'] = count
    ingredients_df['link'] = all_url_filterd[n]
    ingredients_df['source'] = 'all'
    all_ingredients_df = pd.concat([all_ingredients_df, ingredients_df], ignore_index=True)

for n in range(n_two):
    count +=1
    ingredients_df = pd.DataFrame(all_ingredients[n])
    ingredients_df['id'] = count
    ingredients_df['link'] = all_url_filterd[n]
    ingredients_df['source'] = 'two'
    all_ingredients_df = pd.concat([all_ingredients_df, ingredients_df], ignore_index=True)


all_ingredients_df.to_csv('recipes_data.csv', index=False)


In [14]:
all_ingredients_df

,id,source,link,title,description
0,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,olive oil
1,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,chicken breasts
2,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,butter
3,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,garlic
4,1,bbc,https://www.bbc.co.uk/food/recipes/chicken_and...,Chicken and tomato pasta,rosemary
...,...,...,...,...,...
16721,1513,two,https://www.allrecipes.com/creamy-cajun-potato...,Creamy Cajun Potato Soup Recipe,andouille smoked sausage
16722,1513,two,https://www.allrecipes.com/creamy-cajun-potato...,Creamy Cajun Potato Soup Recipe,vegetable broth
16723,1513,two,https://www.allrecipes.com/creamy-cajun-potato...,Creamy Cajun Potato Soup Recipe,heavy cream
16724,1513,two,https://www.allrecipes.com/creamy-cajun-potato...,Creamy Cajun Potato Soup Recipe,Cajun seasoning


# Saving

In [ ]:
scraped_urls = bbc_scraped_urls + all_scraped_urls + two_scraped_urls
scraped_urls_df = pd.DataFrame({'scraped_urls': scraped_urls})
scraped_urls_df.to_csv('scraped_urls.csv', index=False)

In [ ]:
url_filterd = bbc_url_filterd + all_url_filterd + two_url_filterd
url_filterd_df = pd.DataFrame({'url_filterd': url_filterd})
url_filterd_df.to_csv('url_filterd.csv', index=False)

In [ ]:
all_ingredients_df.to_csv('recipes_data.csv', index=False)